In [ ]:
#!wget https://usdf-rsp-dev.slac.stanford.edu/api/consdbtap/sync?LANG=ADQL&REQUEST=doQuery&QUERY=SELECT+TOP+1000+*+FROM+tap_schema.schemas

In [ ]:
# !pip uninstall -y lsst-rsp
#!pip install -y lsst-rsp @ git+https://github.com/lsst-sqre/lsst-rsp@main

In [3]:
from lsst.rsp import get_tap_service

service = get_tap_service("consdbtap")

In [4]:
print("Checking consdbtap tables from tap_schema...")
schema_query = "SELECT * FROM tap_schema.schemas"
checked_tables = []
schema_results = service.search(schema_query)
for schema_name in schema_results["schema_name"]:
    if schema_name == 'tap_schema':
        continue
    table_query = f"SELECT * FROM tap_schema.tables WHERE schema_name = '{schema_name}'"
    print("TAP_SCHEMA query: " + table_query)
    table_results = service.search(table_query)
    for table_name in table_results["table_name"]:
        if not table_name.startswith("tap_schema."):
            single_table_query = f"SELECT * FROM {table_name}"
            print("Table query: " + single_table_query)
            try:
                print(f"Checking table: {table_name}")
                single_table_results = service.search(f"SELECT * FROM {table_name} LIMIT 1").to_table()
                print(f"Successfully read {len(single_table_results)} rows from: {table_name}" + '\n')
            except Exception as e:
                print(f"An error occurred selecting rows from: {table_name}")
                print(e)
            checked_tables.append(table_name.item())
print("Done checking tables!")

Checking consdbtap tables from tap_schema...
TAP_SCHEMA query: SELECT * FROM tap_schema.tables WHERE schema_name = 'cdb_latiss'
Table query: SELECT * FROM cdb_latiss.ccdexposure
Checking table: cdb_latiss.ccdexposure
Successfully read 1 rows from: cdb_latiss.ccdexposure

Table query: SELECT * FROM cdb_latiss.ccdexposure_camera
Checking table: cdb_latiss.ccdexposure_camera
Successfully read 0 rows from: cdb_latiss.ccdexposure_camera

Table query: SELECT * FROM cdb_latiss.ccdexposure_flexdata
Checking table: cdb_latiss.ccdexposure_flexdata
Successfully read 0 rows from: cdb_latiss.ccdexposure_flexdata

Table query: SELECT * FROM cdb_latiss.ccdexposure_flexdata_schema
Checking table: cdb_latiss.ccdexposure_flexdata_schema
Successfully read 0 rows from: cdb_latiss.ccdexposure_flexdata_schema

Table query: SELECT * FROM cdb_latiss.ccdvisit1
Checking table: cdb_latiss.ccdvisit1
Successfully read 1 rows from: cdb_latiss.ccdvisit1

Table query: SELECT * FROM cdb_latiss.ccdvisit1_quicklook
Chec

In [ ]:
print("Checked tables:\n")
for table_name in checked_tables:
    print(table_name)

In [ ]:
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_pyvo_auth, get_service_url
from pyvo.dal import TAPService

In [ ]:
service = get_tap_service("consdbtap")
assert service is not None
service

In [ ]:
schema_query = "SELECT * FROM tap_schema.schemas"
schema_results = service.search(schema_query)
schema_results["schema_name"]

In [ ]:
# query = 
#schema_results = service.search(query).to_table()
#schema_list = [str(row["schema_name"]) for row in schema_results if row["schema_name"] != "tap_schema"]
#schema_list

In [ ]:
#schema_map = {}
#for schema_name in schema_list:
#    schema_map[schema_name] = []
#    tables = service.search(f"SELECT table_name FROM tap_schema.tables WHERE schema_name = '{schema_name}'")
#    for table_name in tables["table_name"]:
#        schema_map[schema_name].append(str(table_name))
#schema_map

In [ ]:
#for schema_name, table_list in schema_map.items():
#    for table_name in table_list:
#        try:
#            results = service.search(f"SELECT * FROM {table_name} LIMIT 10").to_table()
#            print(f"Successfully read {len(results)} rows from: {table_name}")
#        except Exception as e:
#            print(f"An error occurred selecting rows from: {table_name}")
#            print(e)

In [ ]:
service.search("SELECT * FROM cdb_latiss.ccdvisit1")

In [ ]:
# Standalone code to reproduce TAP connection issue
from lsst.rsp import get_tap_service

service = get_tap_service("consdbtap")

schema_query = "SELECT * FROM tap_schema.schemas"
schema_results = service.search(schema_query)
for schema_name in schema_results["schema_name"]:
    table_query = f"SELECT * FROM tap_schema.tables WHERE schema_name = '{schema_name}'"
    table_results = service.search(table_query)
    for table_name in table_results["table_name"]:
        # if table_name.startswith("cdb_lsstcomcamsim."):
        #    print(f"Skipping known bad schema: cdb_lsstcomcamsim")
        if not table_name.startswith("tap_schema."):
            single_table_query = f"SELECT * FROM {table_name}"
            try:
                print(f"Searching on table: {table_name}")
                single_table_results = service.search(f"SELECT * FROM {table_name} LIMIT 10").to_table()
                print(f"Successfully retrieved {len(single_table_results)} from: {table_name}")
            except Exception as e:
                print(e)

In [ ]:
# This doesn't work. Probably a permissions problem writing to the GCS bucket.
service = get_tap_service("consdbtap")
job = service.submit_job("SELECT * FROM cdb_latiss.ccdexposure LIMIT 10")
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table().to_pandas()
print(len(results))

In [ ]:
service.search("SELECT * FROM cdb_lsstcomcamsim.ccdexposure")

In [ ]:
service.search("SELECT * FROM cdb_startrackerfast.exposure")

<h2>Spatial search</h2> 

Find visits by ra and dec and join to ccdvisit table

In [ ]:
service.search("""SELECT v.visit_id, ccdvisit_id, s_ra, s_dec, cv.s_region
FROM cdb_lsstcam.visit1 v
LEFT JOIN cdb_lsstcam.ccdvisit1 cv ON v.visit_id = cv.visit_id
WHERE CONTAINS(
POINT('ICRS', s_ra, s_dec),
CIRCLE('ICRS', 186.8, 7.0, 0.05)) = 1
""")

In [ ]:
service.search("""SELECT TOP 100 s_ra, s_dec
FROM cdb_lsstcam.ccdvisit1_quicklook""")

In [ ]:
#service.search("""SELECT *
#FROM cdb_lsstcam.ccdvisit1
#WHERE CONTAINS(POINT('ICRS', 186.8, 7.0), POLYGON(ccdvisit1.s_region)) = 1
#""")

In [ ]:
#service.search("""SELECT *
#FROM cdb_lsstcam.ccdvisit1
#WHERE CONTAINS(POINT('ICRS', 186.8, 7.0), REGION(ccdvisit1.s_region)) = 1
#""")

In [ ]:
#service.search("""SELECT *
#FROM cdb_lsstcam.ccdvisit1
#WHERE CONTAINS(POINT('ICRS', 186.8, 7.0), POLYGON(ccdvisit1.s_region)) = 1""")

In [ ]:
service.search("""SELECT TOP 100 s_region FROM cdb_lsstcam.ccdvisit1""")

In [ ]:
#service.search("""SELECT *
#FROM cdb_lsstcam.ccdvisit1
#WHERE CONTAINS(POINT('ICRS', 323.26, -24.9), POLYGON('ICRS', 323.267240, -24.955358, 323.277216, -24.727918, 323.522010, -24.736561, 323.512483, -24.964016)) = 1""")

In [ ]:
service.search("""SELECT *
FROM cdb_lsstcam.visit1
WHERE CONTAINS(POINT('ICRS', 323.26, -24.9), s_region) = 1""")

<h2>Scratch</h2>

In [ ]:
service.search("""
SELECT 
    e.exposure_id AS visit_id,
    e.day_obs AS day,
    e.seq_num AS seq,
    e.group_id
    FROM
    cdb_lsstcam.exposure AS e
    WHERE 
    e.img_type = 'acq'
    AND (e.seq_num BETWEEN 128 AND 138)
""")

In [ ]:
service.search("""
SELECT e.group_id 
FROM cdb_lsstcam.exposure AS e
WHERE e.group_id LIKE '%#%'""")


In [ ]:
service.search("""
SELECT 
    e.exposure_id AS visit_id,
    e.day_obs AS day,
    e.seq_num AS seq,
    e.group_id
    FROM
    cdb_lsstcam.exposure AS e
    WHERE 
    e.img_type = 'acq'
    AND (e.seq_num BETWEEN 128 AND 138)
    AND e.group_id LIKE '%#%'
""")